# Smooth Transition from Static to Kinetic Friction Coefficients
we will model a smooth transition between the static (μs) and kinetic (μk) coefficients of friction using mathematical functions

Reference to GitHub issue (commented out)
https://github.com/ipc-sim/ipc-toolkit/pull/139#issuecomment-2479998982

In [41]:
from sympy import symbols, Piecewise, Eq, Abs, integrate, simplify, expand, cos, pi, Poly, nsimplify, lambdify
import numpy as np
import plotly.graph_objects as go
import plotly
import matplotlib.pyplot as plt
from IPython.display import display

In [42]:
x = symbols('x', real=True)
eps_v = symbols(r'\epsilon_v', positive=True) # Threshold velocity for transition
mu_s, mu_k = symbols(r'\mu_s \mu_k', positive=True) # Static and kinetic friction coefficients

# Friction Mollifier

In [43]:
def plot(*fs, title=None, **subs):
    xs = np.linspace(-1, 1, 201)
    subs = {eps_v: 0.5, mu_s: 1, mu_k: 0.1} | subs
    def ys(f):
        return np.array([f.subs({x: xi} | subs) for xi in xs], dtype=float)
    go.Figure(
        [go.Scatter(x=xs, y=ys(f)) for f in fs],
        layout=dict(
        width=800, height=600, template="none",
        xaxis_title=r'$x$', title=title
        )).show()


In [44]:
def plot_interactive(*fs, title=None, **subs):
    xs = np.linspace(-1, 1, 201)
    
    # Default substitution values with slider-friendly range
    default_subs = {
        eps_v: 0.5,  # 0 to 1 range
        mu_s: 1.0,   # 0.1 to 2.0 range
        mu_k: 0.1    # 0 to 1 range
    }
    
    # Update default substitutions with any provided substitutions
    subs = default_subs | subs

    def ys(f):
        return np.array([f.subs({x: xi} | subs) for xi in xs], dtype=float)
    
    # Create traces for each function
    traces = [go.Scatter(x=xs, y=ys(f), name=f'Function {i+1}') for i, f in enumerate(fs)]
    
    # Create figure with sliders
    fig = go.Figure(data=traces)
    
    # Add sliders for eps_v, mu_s, and mu_k
    fig.update_layout(
        sliders=[
            # Epsilon v slider
            dict(
                active=0,
                currentvalue={"prefix": "ε_v: "},
                pad={"t": 120, "l": 200, "r": 120},
                steps=[dict(
                    method='restyle',
                    args=[{'y': [
                        np.array([f.subs({x: xi, eps_v: step, mu_s: subs[mu_s], mu_k: subs[mu_k]}) for xi in xs], dtype=float) 
                        for f in fs
                    ]}],
                    label=f'{step:.2f}'
                ) for step in np.linspace(0.01, 1, 20)],
                x=0.1,  # Positioning the slider
                xanchor='left',
                y=0,
                yanchor='top'
            ),
            # Static friction coefficient slider
            dict(
                active=0,
                currentvalue={"prefix": "μ_s: "},
                pad={"t": 120, "l": 200, "r": 120}, 
                steps=[dict(
                    method='restyle',
                    args=[{'y': [
                        np.array([f.subs({x: xi, eps_v: subs[eps_v], mu_s: step, mu_k: subs[mu_k]}) for xi in xs], dtype=float) 
                        for f in fs
                    ]}],
                    label=f'{step:.2f}'
                ) for step in np.linspace(0.1, 2.0, 20)],
                x=0.1,  # Positioning the slider
                xanchor='left',
                y=-0.1,
                yanchor='top'
            ),
            # Kinetic friction coefficient slider
            dict(
                active=0,
                currentvalue={"prefix": "μ_k: "},
                pad={"t": 120, "l": 200, "r": 120},
                steps=[dict(
                    method='restyle',
                    args=[{'y': [
                        np.array([f.subs({x: xi, eps_v: subs[eps_v], mu_s: subs[mu_s], mu_k: step}) for xi in xs], dtype=float) 
                        for f in fs
                    ]}],
                    label=f'{step:.2f}'
                ) for step in np.linspace(0.01, 1, 20)],
                x=0.1,  # Positioning the slider
                xanchor='left',
                y=-0.2,
                yanchor='top'
            )
        ],
        width=1200, 
        height=1200,
        template="plotly_white",
        title=title,
        xaxis_title=r'$x$'
    )

    # Improve slider interaction
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                showactive=False,
                x=0.6,
                xanchor="left",
                y=1.20,
                yanchor="top",
                pad={"t": 200},
            )
        ]
    )

    fig.show()

The function 𝑓0(𝑥) approximates the absolute value function ∣x∣ but is differentiable at x=0, avoiding sharp changes in force calculations

In [45]:
def f0(x):
    """Smooth friction mollifier"""
    return x**2 * (1 - x / (3 * eps_v)) / eps_v + eps_v / 3

sym_f0 = Piecewise(
    (Abs(x), Abs(x) >= eps_v),
    (f0(Abs(x)), Abs(x) < eps_v)
)

In [46]:
display(Eq(symbols("f_{0}(x)"), expand(sym_f0)))
plot(sym_f0, title=r'$f_0(x)$')
plot_interactive(sym_f0, title=r'$f_0(x)$')

Eq(f_{0}(x), Piecewise((Abs(x), \epsilon_v <= Abs(x)), (\epsilon_v/3 + x**2/\epsilon_v - x**2*Abs(x)/(3*\epsilon_v**2), True)))

In [47]:
def f1(x):
    """Derivative of f0"""
    return x * (2 - x / eps_v) / eps_v

The derivative 𝑓1(𝑥) represents the rate of change of the friction potential with respect to velocity.

In [48]:
sym_f1 = Piecewise(
    (-1, x < -eps_v),
    (-f1(-x), x <= 0),
    (f1(x), x <= eps_v),
    (1, x > eps_v)
)
display(Eq(symbols("f_{1}(x)"), expand(sym_f1)))
plot(sym_f1, title=r"$f_1(x)$")
plot_interactive(sym_f1, title=r"$f_1(x)$")

Eq(f_{1}(x), Piecewise((-1, \epsilon_v < -x), (2*x/\epsilon_v + x**2/\epsilon_v**2, x <= 0), (2*x/\epsilon_v - x**2/\epsilon_v**2, \epsilon_v >= x), (1, True)))

# Smooth Coefficient of Friction
We can use a polynomial to model a smooth transition from static to kinematic coefficient of friction.

In [49]:
def mu(x):
    """Smooth transition from mu_s to mu_k using a polynomial."""
    # Polynomial-based transition 
    # https://blog.demofox.org/2015/08/08/cubic-hermite-interpolation/
    return (mu_k - mu_s) * (3 * (x / eps_v)**2 - 2 * (x / eps_v)**3) + mu_s

sym_mu = Piecewise(
    (mu_k, Abs(x) >= eps_v),
    (mu(Abs(x)), Abs(x) < eps_v)
)
display(Eq(symbols(r"\mu(x)"), expand(sym_mu)))
plot(sym_mu, title=r"$\mu(x)$")
plot_interactive(sym_mu, title=r"$\mu(x)$")

Eq(\mu(x), Piecewise((\mu_k, \epsilon_v <= Abs(x)), (\mu_s + 3*\mu_k*x**2/\epsilon_v**2 - 3*\mu_s*x**2/\epsilon_v**2 - 2*\mu_k*x**2*Abs(x)/\epsilon_v**3 + 2*\mu_s*x**2*Abs(x)/\epsilon_v**3, True)))

In [38]:
def mu(x):
    # return (mu_k - mu_s) * (0.5 * (-cos(pi * x / eps_v) + 1)) + mu_s
    return (mu_k - mu_s) * (3 * (x / eps_v)**2 - 2 * (x / eps_v)**3) + mu_s

sym_mu = Piecewise(
 (mu_k, abs(x) >= eps_v),
 (mu(abs(x)), abs(x) < eps_v)
)
display(Eq(symbols(r"\mu(x)"), sym_mu))
plot(sym_mu, title=r"$\mu(x)$")
plot_interactive(sym_mu, title=r"$\mu(x)$")

# Smooth Coefficient of Friction Molliifier
We know we want a smooth force mollifier of , so we can integrate this function to get a mollifier of that produces the desired force.
This product represents the friction force considering both the velocity dependence and the changing coefficient of friction.

In [50]:
def mu_f1(x):
    return mu(x) * f1(x)

sym_mu_f1 = Piecewise(
    (-mu_k, x < -eps_v),
    (-mu_f1(-x), x <= 0),
    (mu_f1(x), x <= eps_v),
    (mu_k, x > eps_v)
)
display(Eq(symbols(r"\mu(x) f_1(x)"), expand(sym_mu_f1)))
plot(sym_mu_f1, title=r'$\mu(x) f_1(x)$')
plot_interactive(sym_mu_f1, title=r'$\mu(x) f_1(x)$')

False

The resulting force mollifier is a polynomail in , so we can use sympy to get the exact integral.

In [51]:
mu_f0 = integrate(mu_f1(x), x)

In [39]:
# Constant of integration such that mu_f0(eps_v) = mu_k * eps_v
c = mu_k * eps_v - mu_f0.subs(x, eps_v)
mu_f0 = mu_f0 + c
mu_f0 = simplify(mu_f0)

sym_mu_f0 = Piecewise(
    (mu_k * Abs(x), Abs(x) >= eps_v),
    (mu_f0.subs(x, Abs(x)), Abs(x) < eps_v)
)
display(Eq(symbols(r"\int \mu(x) f_1(x) \mathrm{d}x"), expand(sym_mu_f0)))
plot(sym_mu_f0, title=r'$\int \mu(x) f_1(x) \mathrm{d}x$')
plot_interactive(sym_mu_f0, title=r'$\int \mu(x) f_1(x) \mathrm{d}x$')

False

In [40]:
poly_mu_f0 = Poly(nsimplify(mu_f0), x)
display(poly_mu_f0)

Poly((\mu_k - \mu_s)/(3*\epsilon_v**5)*x**6 + (-7*\mu_k + 7*\mu_s)/(5*\epsilon_v**4)*x**5 + (3*\mu_k - 3*\mu_s)/(2*\epsilon_v**3)*x**4 - \mu_s/(3*\epsilon_v**2)*x**3 + \mu_s/\epsilon_v*x**2 + 17*\epsilon_v*\mu_k/30 - 7*\epsilon_v*\mu_s/30, x, domain='ZZ(\epsilon_v,\mu_k,\mu_s)')